<a href="https://colab.research.google.com/github/cche0214/HuggingFaceLLM/blob/main/05Datasets%E5%BA%93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz

In [ ]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

In [ ]:
!ls

In [ ]:
!gzip -dkv SQuAD_it-*.json.gz

In [ ]:
!ls

In [ ]:
# {
#   "version": "1.1",
#   "data": [
#     {
#       "title": "Super_Bowl_50",
#       "paragraphs": [
#         {
#           "context": "Super Bowl 50 was an American football game...",
#           "qas": [
#             {
#               "id": "56be4db0acb8001400a502ec",
#               "question": "Which NFL team represented the AFC?",
#               "answers": [
#                 {"text": "Denver Broncos", "answer_start": 177}
#               ]
#             }
#           ]
#         }
#       ]
#     }
#   ]
# }
# 嵌套字典，样本也就是数据可能在某个字段中

In [ ]:
# {"text": "I love NLP", "label": 1}
# {"text": "I hate bugs", "label": 0}
# {"text": "Transformers are powerful", "label": 1}
# JSON Lines，每一行都是一个JSON，相当于每一行就是一条数据

In [ ]:
# SQuAD-it是嵌套字典，字段存储在data字段里面，field参数的含义是指定从哪个字段取出样本列表
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

In [ ]:
# 加载本地文件会创建一个带有train标签的DatasetDict对象
squad_it_dataset

In [ ]:
squad_it_dataset["train"]

In [ ]:
# 这一条数据的格式是一段文章+一个问题+一个回答的格式
squad_it_dataset["train"][0]

In [ ]:
# 通过向data_files参数传入一个字典，构造同时包含训练集和验证集的一个DatasetDict对象
data_files = {"train": "SQuAD_it-train.json", "test" : "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

In [ ]:
squad_it_dataset

In [ ]:
# Datasets甚至可以自动解压文件，你可以跳过一开始的命令行解压方式
data_files = {"trian": "SQuAD_it-train.json.gz", "test": "SQuAD_it-test.json.gz"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

In [ ]:
squad_it_dataset

In [ ]:
# 加载远程文件和加载本地文件一样简单，只需要换成url
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

In [ ]:
squad_it_dataset

In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"

In [ ]:
!unzip drugsCom_raw.zip

In [ ]:
!ls

In [ ]:
from datasets import load_dataset
# tsv是vsc的一个变体，它使用制表符作为分隔符而不是逗号，因此可以用csv的处理方式，指定分隔符delimiter
data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

In [ ]:
drug_dataset

In [ ]:
drug_dataset["train"]

In [ ]:
drug_dataset["train"][0]

In [ ]:
drug_dataset["train"][1]

In [ ]:
drug_dataset["train"][100]

In [ ]:
# 可以通过Dataset.shuffle()和Dataset.select()函数创建一个随机的样本
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
drug_sample[:3]

In [ ]:
# 这里是在验证Unnamed:0是不是就是患者的ID，把训练集和验证集的数据长度，和这个字段的唯一值比较，相等就说明其实就是患者ID
# assert 条件 意思是如果条件为True，那就什么都不发生，如果为False，那就立即报错
for split in drug_dataset.keys():
  assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [ ]:
drug_dataset.keys()

In [ ]:
drug_dataset

In [ ]:
len(drug_dataset["train"])

In [ ]:
# 确定了Unnamed:0就是患者ID，可以通过DatasetDict.rename_column()进行重命名
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)

In [ ]:
drug_dataset

In [ ]:
# 特定药物和病症的数量
for split in drug_dataset.keys():
  print(len(drug_dataset[split].unique("drugName")))
  print(len(drug_dataset[split].unique("condition")))

In [ ]:
drug_dataset["train"][0]

In [ ]:
# 有关于这个map函数的设计，我们从他处理的数据结构开始理解，也就是Dataset.map()
# Dataset本质是一个保存了很多dict的集合，比如说下面这个是Dataset的一条数据，是一个dict
# 所以Dataset.map()函数的意思就是，对于Dataset里面的每一条数据，也就是每一个dict，运用给定map函数的函数
# 比如下面定义的函数lowercase_condition(example)，根据上面我们知道，这里的example其实就是Dataset里面的一个样本，也就是一个dict
# map要求你的函数返回值必须是一个dict，并且只需要返回你需要修改或者新增的几个keyvalue就好，其他过程全部交给map函数处理
drug_dataset["train"][0]

In [ ]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}


In [ ]:
# 报错说明confition列存在None，不能转换为小写，因为它们不是字符串
drug_dataset.map(lowercase_condition)

In [ ]:
# 可以把存在None的行直接删掉，用Dataset.filter()删除，类似Dataset.map()
# 类似的意思就是，也需要定义函数，传给filter调用，逐行处理样本
# filter是筛选行，对于每一行，返回True保留这一行，返回False删除这一行
# 因此你写的函数返回必须是一个bool变量
def filter_nones(x):
  return x["condition"] is not None

In [ ]:
# 可以发现两个数据集的规模都变小了
drug_dataset = drug_dataset.filter(filter_nones)
drug_dataset

In [ ]:
drug_dataset = drug_dataset.map(lowercase_condition)

In [ ]:
drug_dataset["train"]["condition"][:3]

In [ ]:
# 定义一个函数统计review评论这一列有多少个单词
# 返回的key不在数据里面的任何一个key，所以他会给每一个数据加上一个新的key
def compute_review_length(example):
  return {"review_length": len(example["review"].split())}

In [ ]:
drug_dataset = drug_dataset.map(compute_review_length)

In [ ]:
drug_dataset["train"][0]

In [ ]:
# 查看极端评论的长度，用Dataset.sort()排序，是升序，也就是从小到大
drug_dataset["train"].sort("review_length")[:3]

In [ ]:
# 删除长度小于三十的评论，用filter函数，不过这里直接用匿名函数更方便
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)

In [ ]:
# 删除之后，数据集的规模又小了一点
drug_dataset

In [ ]:
# 处理评论里面的HTML字符，原理如下
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

In [ ]:
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

In [ ]:
# map()方法的加速，batched=True
# 原本对于输入map()的函数，都是对于一条数据的操作，比如说下面的一个字典
# {"text": "hello", "label": 1}
# 如果调用map()的时候加上batched=True参数，就从对一条样本的操作变成一个列表的操作
# {
#   "text": ["hello", "hi", "good morning", ...],   # 长度 = batch_size
#   "label": [1, 0, 1, ...]
# }
# 竖着看，就是一个样本，然后横着走
# 因此对于map()要调用的函数，他的输入和返回值也要改变，用python的列表推导式返回值
# result = [x.lower() for x in texts]
# 输入就变成了一个batch，他的格式就类似上面的列表推导式，同样的你返回也要返回一个列表推导式
# def lowercase_batch(batch):
#     return {
#         "condition": [c.lower() for c in batch["condition"]]
#     }
# batch现在是一个key value(list)的字典，所以batch["condition"]拿到的是一个list，对这个list里面的元素c（代表一个样本的数据)进行处理之后，再返回列表给"condition"字段

In [ ]:
# 快的一比，batched默认1000，这里的意思就是x是一个batch，就是上面的
# {
#   "text": ["hello", "hi", "good morning", ...],   # 长度 = batch_size
#   "label": [1, 0, 1, ...]
# }
# 从里面找review字段，是一个list，把里面的每一个元素o去掉html字符，在通过列表推导式赋给review这个key，因为加入batch之后，这个key就是value(list)
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

In [ ]:
# 快速tokenizer，一般不指定参数直接下载的都是快速tokenizer
# 比较快速/慢速tokenizer和加不加batch对同一文本的编码速度
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples["review"], truncation=True)

In [ ]:
# 这里教程里面写了很乱，我总结一下
# 在from_pretrained()里面可以通过参数use_fast选择导入的是快速tokenizer还是慢速，默认就是快速的
# tokenizer是可以处理list数据的，就是同时对多条样本数据进行tokenize
# 前面说map函数也可以开启batched=True来进行加速，就是处理list结构的数据
# 所以如果你不用map的batched，那么每次tokenize的都是一条一条样本来的
# 如果开启了，就可以结合tokenizer的特性，对batch进行tokenize，也就是list处理
# 因此快速tokenizer加上batched=True是最快的

In [ ]:
# 快速tokenizer加上Batch=True
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

In [ ]:
# 快速tokenizer加上Batch=Flase
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

In [ ]:
# 快速tokenizer，一般不指定参数直接下载的都是快速tokenizer
# 比较快速/慢速tokenizer和加不加batch对同一文本的编码速度
from transformers import AutoTokenizer

# 慢速
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

def tokenize_function(examples):
  return tokenizer(examples["review"], truncation=True)

In [ ]:
# 慢速tokenizer加上Batch=True
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

In [ ]:
# 慢速tokenizer加上Batch=False
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

In [ ]:
# 快速tokenizer，一般不指定参数直接下载的都是快速tokenizer
# 比较快速/慢速tokenizer和加不加batch对同一文本的编码速度
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples["review"], truncation=True)

In [ ]:
drug_dataset

In [ ]:
# 快速tokenizer加上batch加多进程
# 不推荐在快速tokenizer和batch的情况下使用多进程，没必要
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True, num_proc=8)

In [ ]:
print(type(drug_dataset))
print(drug_dataset)

In [ ]:
# 从一个样本中创建几个训练特征
def tokenize_and_split(examples):
  return tokenizer(
      examples["review"],
      truncation=True,
      max_length=128,
      # 返回全部文本块而不是丢弃只保留第一个，按128切分了
      return_overflowing_tokens=True
  )

In [ ]:
# 上面是在一个数据上的测试结果
# 一个样本变成了两个特征，一个长度128，一个长度49
[len(inp) for inp in result["input_ids"]]

In [ ]:
result

In [ ]:
# 详细解释一下下面的报错
# Dataset是一个包含了很多字典的表，类似
# 行0: {review: "...", label: 1}
# 行1: {review: "...", label: 0}
# 行2: {review: "...", label: 1}
# ...
# 选择batched=True的时候，map函数看到的就变成
# batch = {
#   "review": ["r0", "r1", "r2", ..., "r999"],   # 长度 = N
#   "label":  [  1,    0,    1,  ...,    0   ]   # 长度 = N
# }
# 你写的map函数，就是给你一个batch，我需要返回一个dict，map函数会把我返回的dict和原来的dataset拼在一起
# 所以加入我返回了一个新列的列表，就是key:value(list)是一一对应的，比如
# return {
#   "new_feature": ["x0", "x1", "x2", ..., "x999"]
# }
# 那么最后得到的Dataset就是正常的，比如
# 行0: {review: r0, new_feature: x0}
# 行1: {review: r1, new_feature: x1}
# ...
# 所以现在的问题是，由于前面选择了不丢弃超出长度的token，导致map调用的函数返回的长度和dataset的数据条目不匹配了，也就是
# return {
#   "input_ids": [...长度1463...],
#   "attention_mask": [...长度1463...]
# }
# 所以map函数想把我返回的input_ids和attention_mask拼接回去Dataset的时候报错
# 明明一批应该只有1000个，对应1000个新属性：新值，为什么现在又1463个

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

In [ ]:
# 问题就是我们想混合两个长度不同的数据集
# 那么解决办法就是，要么删除旧数据集的列（其实就是丢弃原来的数据），要么它们和新数据集里面的尺寸相同
# 这里实现前者
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)

In [ ]:
# 处理之后的新数据集比原始的要大，因为我们没有丢弃被拆分的token
len(tokenized_dataset["train"]),len(drug_dataset["train"])

In [ ]:
# 也可以选择把旧的列变成和新列保持相同大小
# 用overflow_to_sample_mapping，给出新特征索引到它源自的样本索引的映射
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # 提取新旧索引之间的映射
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

In [ ]:
tokenized_dataset

In [ ]:
# Datasets和DataFrames的相互转换
drug_dataset

In [ ]:
drug_dataset.set_format("pandas")

In [ ]:
drug_dataset

In [ ]:
drug_dataset["train"][:3]

In [ ]:
# drug_dataset["train"]依旧是Dataset格式，set_format只改变了__getitem__()方法的返回格式

drug_dataset["train"]

In [ ]:
# 需要对整个数据集切片才可以获得pandas.DataFrame格式的数据集
train_df = drug_dataset["train"][:]

In [ ]:
train_df

In [ ]:
frequencies = (
    train_df["condition"] # 找到这个表的condition这一列
    .value_counts() # 统计这个列里面每个值出现的次数
    .to_frame() # 把上面统计的结果转换成表
    .reset_index() # 给这个表加上index索引，就是下面的样子
    .rename(columns={"index": "condition", "count": "frequency"}) # 给表的列改名
)
frequencies.head()

In [ ]:
# Dataset.from_pandas()创建一个新的Dataset对象
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

In [ ]:
drug_dataset

In [ ]:
drug_dataset["train"]

In [ ]:
train_df

In [ ]:
# 试试看！
drug_avg = (
    train_df
    .groupby("drugName")["rating"]
    .mean()
    .reset_index()
)
drug_avg.head()

In [ ]:
drug_avg_dataset = Dataset.from_pandas(drug_avg)
drug_avg_dataset

In [ ]:
# 重置成Arrow
drug_dataset.reset_format()

In [ ]:
# 创建验证集合
drug_dataset_clean = drug_dataset["train"].train_test_split(
    train_size = 0.8,
    seed = 42
)

In [ ]:
drug_dataset_clean

In [ ]:
# 改名，把test改成validation
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")

In [ ]:
drug_dataset_clean

In [ ]:
drug_dataset_clean["test"] = drug_dataset["test"]

In [ ]:
drug_dataset_clean

In [ ]:
# 保存数据集，可以保存Arrow，CSV，JSON格式的
# Arrow格式
drug_dataset_clean.save_to_disk("drug_reviews")

In [ ]:
!ls

In [ ]:
!ls ./drug_reviews/

In [ ]:
!ls ./drug_reviews/test

In [ ]:
# load_from_disk()从磁盘读取数据
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("drug_reviews")
drug_dataset_reloaded